In [2]:
import numpy as np
import chainer
from chainer import cuda, Function, \
        report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

### <font color='blue'>変数と偏微分のやり方</font>

In [3]:
x1 = Variable(np.array([1]).astype(np.float32))
x2 = Variable(np.array([2]).astype(np.float32))
x3 = Variable(np.array([3]).astype(np.float32))

In [4]:
z = (x1-2*x2-1)**2 + (x2*x3 - 1)**2 + 1

In [5]:
z.data

array([ 42.], dtype=float32)

In [6]:
z.backward()

In [7]:
x1.grad

array([-8.], dtype=float32)

In [8]:
x2.grad

array([ 46.], dtype=float32)

In [9]:
x3.grad

array([ 20.], dtype=float32)

### <font color='blue'>sin, sigmoid関数</font>

In [10]:
x = Variable(np.array([-1], dtype=np.float32))

In [11]:
F.sin(x).data # sin関数

array([-0.84147096], dtype=float32)

In [12]:
F.sigmoid(x).data # sigmoid関数

array([ 0.2689414], dtype=float32)

### <font color='blue'>cos関数とsin関数で微分の確認</font>

In [13]:
x = Variable(np.array([-0.5], dtype=np.float32))

In [14]:
z = F.cos(x)

In [15]:
z.data

array([ 0.87758255], dtype=float32)

In [16]:
z.backward()

In [17]:
x.grad

array([ 0.47942555], dtype=float32)

In [18]:
((-1) * F.sin(x)).data

array([ 0.47942555], dtype=float32)

### <font color='blue'>変数が多次元</font>

In [19]:
x = Variable(np.array([-1, 0, 1], dtype=np.float32))

In [20]:
z = F.sin(x)

In [21]:
z.grad = np.ones(3, dtype=np.float32)

In [22]:
z.backward()

In [23]:
x.grad

array([ 0.54030228,  1.        ,  0.54030228], dtype=float32)

### <font color='blue'>links</font>

自分が考えたモデルをlinksとfunctionで、単純に組み合わせることができたら<br>
それで、Chainerのプログラムは完成である。<br>

### <font color='blue'>重みとバイアス</font>

l層の入力が３次元、l+1層の次元が４次元となっている。

In [24]:
h = L.Linear(3, 4)

In [25]:
h.W.data

array([[ 0.18997002,  0.48362765, -0.27716058],
       [-0.06260166,  0.22578894, -0.4718852 ],
       [ 0.40706906, -0.52103478,  0.97875464],
       [-0.30818981, -0.44233015,  0.22494164]], dtype=float32)

In [26]:
h.b.data

array([ 0.,  0.,  0.,  0.], dtype=float32)

In [30]:
x = Variable(np.array(range(6)).astype(np.float32).reshape(2,3))

In [31]:
x.data.shape

(2, 3)

In [33]:
x.data

array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.]], dtype=float32)

In [34]:
y = h(x)

In [35]:
y.data

array([[-0.07069352, -0.71798146,  1.43647456,  0.00755313],
       [ 1.11861777, -1.64407527,  4.03084087, -1.56918192]], dtype=float32)

In [36]:
y.data.shape

(2, 4)

### <font color='blue'>正しく計算できているかをみる</font>

In [37]:
w = h.W.data

In [38]:
x0 = x.data

In [39]:
x0.dot(w.T) + h.b.data

array([[-0.07069352, -0.71798146,  1.43647456,  0.00755313],
       [ 1.11861777, -1.64407527,  4.03084087, -1.56918192]], dtype=float32)

### <font color='blue'>Chainクラス</font>

In [40]:
class MyChain(Chain):
    def __init__(self):
        super(MyChain, self).__init__(
            l1 = L.Linear(4,3), # 入力層
            l2 = L.Linear(3,3), #　出力層
        )
        
    def __call__(self, x, y):
        fv = self.fwd(x, y) # 順伝播
        loss = F.mean_squared_error(fv, y) # 誤差算出
        return loss # 誤差を返す
        
    def fwd(self, x, y):
        return F.sigmoid(self.l1(x)) # シグモイド関数に入れて出力

In [41]:
model = MyChain()
optimizer = optimizers.SGD()
optimizer.setup(model)